In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import json

# These commands below set some options for pandas and to have matplotlib show the charts in the notebook
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

# Load the data
# We have this defaulted to the folder OUTSIDE of your repo - please change it as needed
population = pd.read_csv('Population by Age and Sex - US, States, Counties.csv')
with open('legislators-historical.json', 'r') as file:
    file_data = json.load(file)
rows = []
for person in file_data:
    base_info = {
        "bioguide_id": person["id"].get("bioguide"),
        "govtrack_id": person["id"].get("govtrack"),
        "icpsr_id": person["id"].get("icpsr"),
        "wikipedia": person["id"].get("wikipedia"),
        "wikidata": person["id"].get("wikidata"),
        "first_name": person["name"]["first"],
        "last_name": person["name"]["last"],
        "gender": person["bio"].get("gender"),
        "birthday": person["bio"].get("birthday"),
    }
    for term in person["terms"]:
        row = base_info.copy()
        row.update({
            "type": term["type"],
            "start": term["start"],
            "end": term["end"],
            "state": term["state"],
            "district": term.get("district"),
            "party": term.get("party"),
            "class": term.get("class"),
        })
        rows.append(row)

# # Convert to DataFrame
legislators = pd.DataFrame(rows)

# Note - for now, it is okay to ignore the warning about mixed types.

In [13]:
# Display the DataFrame

# Convert term start & end to datetimes
legislators['start'] = pd.to_datetime(legislators['start'])
legislators['end'] = pd.to_datetime(legislators['end'])

# Filter to starting on or after 2000
leg_filtered_df = legislators[legislators['start'].dt.year >= 1900] 

leg_filtered_df[leg_filtered_df["last_name"] == ""]


,bioguide_id,govtrack_id,icpsr_id,wikipedia,wikidata,first_name,last_name,gender,birthday,type,start,end,state,district,party,class


In [35]:
##Gabby test cell
filtered_pop = population[population['Description'] == 'U.S.']

# Remove unnecessary columns
filtered_pop = filtered_pop.drop(['IBRC_Geo_ID', 'Statefips', 'Countyfips'], axis=1)

# Function to get percentages of populations
def percent(row, name):
    base = row['Total Population']
    return row[name]/base

for c in filtered_pop.columns:
    if 'population' in c.lower():
        filtered_pop['% ' + c] = filtered_pop.apply(lambda row: percent(row, c), axis = 1)
#filtered_pop['0-4 Percent'] = filtered_pop.apply(lambda row: percent(row, "Population 0-4"), axis = 1)
filtered_pop.columns
filtered_pop

,Description,Year,Total Population,Population 0-4,Population 5-17,Population 18-24,Population 25-44,Population 45-64,Population 65+,Population Under 18,...,% Population 5-17,% Population 18-24,% Population 25-44,% Population 45-64,% Population 65+,% Population Under 18,% Population 18-54,% Population 55+,% Male Population,% Female Population
0,U.S.,2000,"282,162,411.00","19,178,293.00","53,197,896.00","27,315,274.00","84,973,340.00","62,428,040.00","35,069,568.00","72,376,189.00",...,0.19,0.10,0.30,0.22,0.12,0.26,0.53,0.21,0.49,0.51
1,U.S.,2001,"284,968,955.00","19,298,217.00","53,372,958.00","27,992,652.00","84,523,274.00","64,491,563.00","35,290,291.00","72,671,175.00",...,0.19,0.10,0.30,0.23,0.12,0.26,0.53,0.21,0.49,0.51
2,U.S.,2002,"287,625,193.00","19,429,192.00","53,507,265.00","28,480,708.00","83,990,295.00","66,695,526.00","35,522,207.00","72,936,457.00",...,0.19,0.10,0.29,0.23,0.12,0.25,0.53,0.22,0.49,0.51
3,U.S.,2003,"290,107,933.00","19,592,446.00","53,508,312.00","28,916,746.00","83,398,001.00","68,828,899.00","35,863,529.00","73,100,758.00",...,0.18,0.10,0.29,0.24,0.12,0.25,0.53,0.22,0.49,0.51
4,U.S.,2004,"292,805,298.00","19,785,885.00","53,511,850.00","29,302,179.00","83,066,831.00","70,935,234.00","36,203,319.00","73,297,735.00",...,0.18,0.10,0.28,0.24,0.12,0.25,0.53,0.22,0.49,0.51
5,U.S.,2005,"295,516,599.00","19,917,400.00","53,606,269.00","29,441,546.00","82,764,185.00","73,137,401.00","36,649,798.00","73,523,669.00",...,0.18,0.10,0.28,0.25,0.12,0.25,0.52,0.23,0.49,0.51
6,U.S.,2006,"298,379,912.00","19,938,883.00","53,818,831.00","29,602,839.00","82,638,980.00","75,216,272.00","37,164,107.00","73,757,714.00",...,0.18,0.10,0.28,0.25,0.12,0.25,0.52,0.23,0.49,0.51
7,U.S.,2007,"301,231,207.00","20,125,962.00","53,893,443.00","29,808,025.00","82,509,693.00","77,068,373.00","37,825,711.00","74,019,405.00",...,0.18,0.10,0.27,0.26,0.13,0.25,0.52,0.24,0.49,0.51
8,U.S.,2008,"304,093,966.00","20,271,127.00","53,833,475.00","30,194,274.00","82,399,959.00","78,617,510.00","38,777,621.00","74,104,602.00",...,0.18,0.10,0.27,0.26,0.13,0.24,0.52,0.24,0.49,0.51
9,U.S.,2009,"306,771,529.00","20,244,518.00","53,889,649.00","30,530,346.00","82,211,153.00","80,272,688.00","39,623,175.00","74,134,167.00",...,0.18,0.10,0.27,0.26,0.13,0.24,0.51,0.24,0.49,0.51


In [36]:
##Ambro Test cell
def load_json(file_name: str):
    with open(file_name, 'r') as file:
        file_data = json.load(file)
    rows = []
    for person in file_data:
        base_info = {
            "bioguide_id": person["id"].get("bioguide"),
            "govtrack_id": person["id"].get("govtrack"),
            "icpsr_id": person["id"].get("icpsr"),
            "wikipedia": person["id"].get("wikipedia"),
            "wikidata": person["id"].get("wikidata"),
            "first_name": person["name"]["first"],
            "last_name": person["name"]["last"],
            "gender": person["bio"].get("gender"),
            "birthday": person["bio"].get("birthday"),
        }
        for term in person["terms"]:
            row = base_info.copy()
            row.update({
                "type": term["type"],
                "start": term["start"],
                "end": term["end"],
                "state": term["state"],
                "district": term.get("district"),
                "party": term.get("party"),
                "class": term.get("class"),
            })
            rows.append(row)

    # Convert to DataFrame
    legislators = pd.DataFrame(rows)
    return legislators
incumbent = load_json('legislators-current.json')
non_incumbent = load_json('legislators-historical.json')

In [22]:
##Uma test cell

In [ ]:
##Chad test cell